In [1]:
import glob
import os
import sys
sys.path.append("..")
import pandas as pd
from pandas.io.parsers import read_csv

from utils.configuration import Configuration

config = Configuration()
def add_file_name(csv, path):
    file_name = os.path.basename(path).split('.')[0]
    csv['config'] = file_name.split('result_')[1]
    return csv

In [2]:
def read_results():
    desc_map_files = []
    for path in glob.glob('../'+config.results_dir + "/*.csv"):
        csv = read_csv(path, encoding='latin-1')
        csv = add_file_name(csv, path)
        desc_map_files.append(csv)
    results= pd.concat(desc_map_files, axis=0, ignore_index=True)
    results.drop('target_app', inplace=True, axis=1)
    results.drop('src_app', inplace=True, axis=1)
    results.fillna(0, inplace=True)
    return results

In [3]:
def config_average_performance(results):
    return results.groupby('config').mean().sort_values(by = ['accuracy'], ascending=False)

In [4]:
results = read_results()
agg_results = config_average_performance(results).sort_values(by = ['f1_score'], ascending=False)

In [5]:
# config_stat = read_csv('../'+config.analyse_dir + '/configs_status.csv')
# config_stat.set_index('config', inplace=True)
# final_result = config_stat.join(agg_results)
final_result = agg_results
final_result

,tp,tn,fp,fn,effort,accuracy,precision,recall,f1_score,reduction
config,,,,,,,,,,
wm_googleplay_craftdroid_intersection,2.064516,0.870968,1.451613,1.225806,2.145161,0.610785,0.583295,0.618548,0.591094,0.537903
jaccard_empty_craftdroid_atm,2.080645,0.870968,1.580645,1.209677,2.161290,0.606776,0.580876,0.615591,0.588447,0.523925
jaccard_empty_custom_atm,2.049180,0.885246,1.606557,1.245902,2.163934,0.588526,0.567116,0.599727,0.571574,0.512933
wm_blogs_craftdroid_intersection,1.950820,0.885246,1.622951,1.344262,2.278689,0.591568,0.551483,0.594536,0.567241,0.515027
use_standard_craftdroid_craftdroid,2.000000,0.870968,1.741935,1.290323,2.274194,0.585999,0.547632,0.596774,0.565956,0.506720
...,...,...,...,...,...,...,...,...,...,...
w2v_blogs_atm_union,1.639344,0.885246,1.967213,1.655738,2.655738,0.504202,0.473992,0.493989,0.473095,0.406922
fast_googleplay_atm_union,1.629032,0.870968,2.048387,1.661290,2.693548,0.499317,0.465047,0.491398,0.468162,0.396237
wm_googleplay_atm_union,1.590164,0.885246,1.983607,1.704918,2.688525,0.499536,0.464065,0.492077,0.468159,0.404098


In [6]:
configs_mrr = read_csv('../' + config.analyse_dir + '/mrr.csv')
configs_mrr['config'] = configs_mrr[['word_embedding', 'training_set', 'algorithm','descriptors']].agg('_'.join, axis=1)
configs_mrr.drop(columns=['descriptors','word_embedding', 'training_set', 'algorithm'], inplace=True)
configs_mrr.set_index('config', inplace=True)
final_result = final_result.join(configs_mrr).sort_values(by = ['f1_score'], ascending=False)
# configs_mrr.loc[configs_mrr['config'] == 'glove_android_atm_union']
final_result.to_csv('../' + config.analyse_dir + '/final_results.csv')